In [15]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
import pandas as pd

In [4]:
load_dotenv()

True

In [7]:
DB_CONFIG ={
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT'),
    'database': os.getenv('DB_NAME'),
    'username': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD')
}

In [13]:
try:
    connection_string = (
        f"mysql+pymysql://{DB_CONFIG['username']}:{DB_CONFIG['password']}"
        f"@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
    )   
    engine = create_engine(connection_string)

    with engine.connect() as conn:
        print("conectado ao mysql")

        result = conn.execute(text("SELECT NOW() AS hora_atual"))
        hora = result.fetchone()
        print(f"hora do servidor: {hora[0]}")
except Exception as e:
    print(f"Erro {e}")

conectado ao mysql
hora do servidor: 2025-11-22 23:54:50


In [14]:
query = "SELECT * FROM customers"

In [17]:
df_customers = pd.read_sql(query, engine)
df_customers

,customer_id,nome,age,city
0,1,Bruno Silva,21,Los Angeles
1,2,Caio Jorge,25,San Diego
2,3,Camila Vitória,18,Los Angeles
3,4,Vicente Marques,71,Los Angeles
4,5,Guilherme Freitas,25,New York
5,6,Carlos Magno,48,New York


In [20]:
query2 = " SELECT * FROM orders_details;"

In [21]:
df_order = pd.read_sql(query2, engine)
df_order

,nome,city,product_name,category,quantity,price,order_date,total_value
0,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5
1,Bruno Silva,Los Angeles,ketchup,pantry,50,5.99,2025-11-16,299.5
2,Caio Jorge,San Diego,Apple Juice,produce,30,2.75,2025-11-15,82.5
3,Bruno Silva,Los Angeles,Apple Juice,produce,50,2.75,2025-11-09,137.5
4,Carlos Magno,New York,shampoo,personal care,100,3.99,2025-11-12,399.0
5,Camila Vitória,Los Angeles,Past,pantry,20,1.99,2025-11-01,39.8
6,Camila Vitória,Los Angeles,Apple Juice,produce,10,2.75,2025-11-04,27.5
7,Guilherme Freitas,New York,Soda,produce,10,1.00,2025-11-19,10.0
8,Carlos Magno,New York,milk,dairy,100,1.99,2025-11-21,199.0


In [ ]:
vendas_por_cliente = df_order.groupby("nome")["total_value"].sum().reset_index()
vendas_por_cliente


,nome,total_value
0,Bruno Silva,736.5
1,Caio Jorge,82.5
2,Camila Vitória,67.3
3,Carlos Magno,598.0
4,Guilherme Freitas,10.0


In [31]:
vendas_por_produto = df_order.groupby("product_name")["total_value"].sum().reset_index()
vendas_por_produto

,product_name,total_value
0,Apple Juice,247.5
1,Past,39.8
2,Soda,10.0
3,ketchup,599.0
4,milk,199.0
5,shampoo,399.0


In [32]:
quantidade_por_categoria = df_order.groupby("category")["quantity"].sum().reset_index()
quantidade_por_categoria

,category,quantity
0,dairy,100
1,pantry,120
2,personal care,100
3,produce,100


In [38]:
vendas_por_dia = df_order.groupby("order_date")["total_value"].sum().reset_index()
vendas_por_dia.sort_values(by='total_value', ascending=False)

,order_date,total_value
5,2025-11-16,599.0
3,2025-11-12,399.0
7,2025-11-21,199.0
2,2025-11-09,137.5
4,2025-11-15,82.5
0,2025-11-01,39.8
1,2025-11-04,27.5
6,2025-11-19,10.0
